In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=osTVH909K1G3y2RMeSanJrjDNlj0Nh90WQ07q8Wmy8Y&tc=NCfqjMuPVbCYrScWfd4fXVMw3S6VX5vzsVV_QC-kdCs&cc=UAFzFV0aBjmWp_-bs5dqD80EWSH4cGMFblPUHYnkvMY

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AWtgzh7K1Zl2q6piwAt4K0H8s2LmfiqwM6cjJWKwPIhrsemHyMW-N2fTGVU

Successfully saved authorization token.


In [ ]:
import pandas as pd
import altair as alt
import numpy as np
import folium

In [ ]:
def create_reduce_region_function(geometry,
                                  reducer=ee.Reducer.mean(),
                                  scale=1000,
                                  crs='EPSG:4326',
                                  bestEffort=True,
                                  maxPixels=1e13,
                                  tileScale=4):
  """Creates a region reduction function.

  Creates a region reduction function intended to be used as the input function
  to ee.ImageCollection.map() for reducing pixels intersecting a provided region
  to a statistic for each image in a collection. See ee.Image.reduceRegion()
  documentation for more details.

  Args:
    geometry:
      An ee.Geometry that defines the region over which to reduce data.
    reducer:
      Optional; An ee.Reducer that defines the reduction method.
    scale:
      Optional; A number that defines the nominal scale in meters of the
      projection to work in.
    crs:
      Optional; An ee.Projection or EPSG string ('EPSG:5070') that defines
      the projection to work in.
    bestEffort:
      Optional; A Boolean indicator for whether to use a larger scale if the
      geometry contains too many pixels at the given scale for the operation
      to succeed.
    maxPixels:
      Optional; A number specifying the maximum number of pixels to reduce.
    tileScale:
      Optional; A number representing the scaling factor used to reduce
      aggregation tile size; using a larger tileScale (e.g. 2 or 4) may enable
      computations that run out of memory with the default.

  Returns:
    A function that accepts an ee.Image and reduces it by region, according to
    the provided arguments.
  """

  def reduce_region_function(img):
    """Applies the ee.Image.reduceRegion() method.

    Args:
      img:
        An ee.Image to reduce to a statistic by region.

    Returns:
      An ee.Feature that contains properties representing the image region
      reduction results per band and the image timestamp formatted as
      milliseconds from Unix epoch (included to enable time series plotting).
    """

    stat = img.reduceRegion(
        reducer=reducer,
        geometry=geometry,
        scale=scale,
        crs=crs,
        bestEffort=bestEffort,
        maxPixels=maxPixels,
        tileScale=tileScale)

    return ee.Feature(geometry, stat).set({'millis': img.date().millis()}).set({'DOY':img.date().getRelative('day', 'year')})
  return reduce_region_function

In [ ]:
# Define a function to transfer feature properties to a dictionary.
def fc_to_dict(fc):
  prop_names = fc.first().propertyNames()
  prop_lists = fc.reduceColumns(
      reducer=ee.Reducer.toList().repeat(prop_names.size()),
      selectors=prop_names).get('list')

  return ee.Dictionary.fromLists(prop_names, prop_lists)

In [ ]:
# country you are interested in

countries = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level0")

def GetCountryGeometry(country_name):
  return countries.filter(ee.Filter.eq('ADM0_NAME', country_name)).first().geometry()

country_name = 'South Africa'  #select country name
geom = GetCountryGeometry(country_name)
aoi = ee.Feature(geom)

country_name = country_name.replace(' ','')
print('country : ', country_name)

country :  SouthAfrica


In [ ]:
precipitation = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY").sort('system:time_start').filterBounds(geom)

In [ ]:
today = ee.Date(pd.to_datetime('today'))
date_range = ee.DateRange(today.advance(-20, 'years'), today)
precipitation = precipitation.filterDate(date_range).select('precipitation')

In [ ]:
precipitation.size().getInfo()

7276

In [ ]:
precipitation.first().getInfo()

{'type': 'Image',
 'bands': [{'id': 'precipitation',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [7200, 2000],
   'crs': 'EPSG:4326',
   'crs_transform': [0.05, 0, -180, 0, -0.05, 50]}],
 'version': 1515006749270432,
 'id': 'UCSB-CHG/CHIRPS/DAILY/20030302',
 'properties': {'system:time_start': 1046563200000,
  'system:footprint': {'type': 'LinearRing',
   'coordinates': [[-180, -90],
    [180, -90],
    [180, 90],
    [-180, 90],
    [-180, -90]]},
  'system:time_end': 1046649600000,
  'system:asset_size': 4757876,
  'system:index': '20030302'}}

In [ ]:
average_list = ee.List([])
def AveragePer8(img):
  date = ee.Date(ee.List(img.get('system:time_start')))
  date_range = ee.DateRange(date.advance(-8, 'days'), date)
  imagecollection = precipitation.filterDate(date_range)

  return imagecollection.mean().set({'millis': img.date().millis()})

In [ ]:
listOfImages = precipitation.toList(precipitation.size())
average_list = ee.List([])
for i in range(1, int(precipitation.size().getInfo())):
  img = ee.Image(listOfImages.get(i))
  avg_img = AveragePer8(img)
  average_list.add(ee.Image(avg_img))

In [ ]:
average_list.size().getInfo()

0

In [ ]:
precipitation_avarage = precipitation.map(AveragePer8)

In [ ]:
precipitation_avarage = ee.ImageCollection.fromImages(precipitation_avarage)

In [ ]:
precipitation_avarage

In [ ]:
reduce_precipitation = create_reduce_region_function(
    geometry=geom, reducer=ee.Reducer.mean(), scale=250, crs='EPSG:4326')

Precipitation_stat_fc = ee.FeatureCollection(precipitation.map(reduce_precipitation)).filter(ee.Filter.notNull(precipitation.first().bandNames()))

In [ ]:
Precipitation_stat_fc = Precipitation_stat_fc.filter(ee.Filter.Or(
    ee.Filter.eq('DOY', 0),
    ee.Filter.eq('DOY', 8),
    ee.Filter.eq('DOY', 16),
    ee.Filter.eq('DOY', 24),
    ee.Filter.eq('DOY', 32),
    ee.Filter.eq('DOY', 40),
    ee.Filter.eq('DOY', 48),
    ee.Filter.eq('DOY', 56),
    ee.Filter.eq('DOY', 64),
    ee.Filter.eq('DOY', 72),
    ee.Filter.eq('DOY', 80),
    ee.Filter.eq('DOY', 88),
    ee.Filter.eq('DOY', 96),
    ee.Filter.eq('DOY', 104),
    ee.Filter.eq('DOY', 112),
    ee.Filter.eq('DOY', 120),
    ee.Filter.eq('DOY', 128),
    ee.Filter.eq('DOY', 136),
    ee.Filter.eq('DOY', 144),
    ee.Filter.eq('DOY', 152),
    ee.Filter.eq('DOY', 160),
    ee.Filter.eq('DOY', 168),
    ee.Filter.eq('DOY', 176),
    ee.Filter.eq('DOY', 184),
    ee.Filter.eq('DOY', 192),
    ee.Filter.eq('DOY', 200),
    ee.Filter.eq('DOY', 208),
    ee.Filter.eq('DOY', 216),
    ee.Filter.eq('DOY', 224),
    ee.Filter.eq('DOY', 232),
    ee.Filter.eq('DOY', 240),
    ee.Filter.eq('DOY', 248),
    ee.Filter.eq('DOY', 256),
    ee.Filter.eq('DOY', 264),
    ee.Filter.eq('DOY', 272),
    ee.Filter.eq('DOY', 280),
    ee.Filter.eq('DOY', 288),
    ee.Filter.eq('DOY', 296),
    ee.Filter.eq('DOY', 304),
    ee.Filter.eq('DOY', 312),
    ee.Filter.eq('DOY', 320),
    ee.Filter.eq('DOY', 328),
    ee.Filter.eq('DOY', 336),
    ee.Filter.eq('DOY', 344),
    ee.Filter.eq('DOY', 352),
    ee.Filter.eq('DOY', 360)
))

In [ ]:
asset_path = 'projects/project_name/' + str(country_name) + '_Precipitation_fc_ts_vis_with_altair'

task = ee.batch.Export.table.toAsset(
    collection=Precipitation_stat_fc,
    description='Precipitation_stat_fc export',
    assetId=asset_path)

task.start()   # It takes some time to make a data

In [ ]:
filename = country_name
task = ee.batch.Export.table.toDrive(
  collection= Precipitation_stat_fc,
  description= 'file2',
  fileFormat= 'CSV',
  folder='Precipitation'
)
#task.start()   # It takes some time to make a data

In [ ]:
precipitation_stat_fc = ee.FeatureCollection('projects/ee-w51355846/assets/GambiaPrecipitation')

In [ ]:
precipitation_dict = fc_to_dict(precipitation_stat_fc).getInfo()

In [ ]:
print(type(precipitation_dict), '\n')
for prop in precipitation_dict.keys():
    print(prop + ':', precipitation_dict[prop][0:3] + ['...'])

<class 'dict'> 

DOY: [0, 0, 0, '...']
millis: [1546300800000, 1293840000000, 1325376000000, '...']
precipitation: [0.0021515116737850157, 0.00017078185702264665, 0.017809911233836632, '...']
system:index: ['000000000000000000d0', '0000000000000000097a', '00000000000000000ae7', '...']


In [ ]:
precipitation_df = pd.DataFrame(precipitation_dict)
display(precipitation_df)
print(precipitation_df.dtypes)

,DOY,millis,precipitation,system:index
0,0,1546300800000,0.002152,000000000000000000d0
1,0,1293840000000,0.000171,0000000000000000097a
2,0,1325376000000,0.017810,00000000000000000ae7
3,0,1356998400000,0.012265,00000000000000000c55
4,0,1388534400000,0.000002,00000000000000000dc2
...,...,...,...,...
7271,255,1094947200000,0.300554,00000000000000001644
7272,255,1126569600000,1.969833,000000000000000017b2
7273,255,1158105600000,2.503198,0000000000000000191f
7274,255,1189641600000,3.391113,00000000000000001a8c


DOY                int64
millis             int64
precipitation    float64
system:index      object
dtype: object


In [ ]:
# Function to add date variables to DataFrame.
def add_date_info(df):
  df['Timestamp'] = pd.to_datetime(df['millis'], unit='ms')
  df['Year'] = pd.DatetimeIndex(df['Timestamp']).year
  df['Month'] = pd.DatetimeIndex(df['Timestamp']).month
  df['Day'] = pd.DatetimeIndex(df['Timestamp']).day
  return df

In [ ]:
precipitation_df = add_date_info(precipitation_df)
precipitation_df.head(5)

,DOY,millis,precipitation,system:index,Timestamp,Year,Month,Day
0,0,1546300800000,0.002152,000000000000000000d0,2019-01-01,2019,1,1
1,0,1293840000000,0.000171,0000000000000000097a,2011-01-01,2011,1,1
2,0,1325376000000,0.017810,00000000000000000ae7,2012-01-01,2012,1,1
3,0,1356998400000,0.012265,00000000000000000c55,2013-01-01,2013,1,1
4,0,1388534400000,0.000002,00000000000000000dc2,2014-01-01,2014,1,1


In [ ]:
precipitation_df = precipitation_df.drop(columns=['millis', 'system:index'])
precipitation_df = precipitation_df.sort_values(by='Timestamp').reset_index(drop=True)
precipitation_df.head(5)

,DOY,precipitation,Timestamp,Year,Month,Day
0,60,0.000000,2003-03-02,2003,3,2
1,61,0.000000,2003-03-03,2003,3,3
2,62,0.000000,2003-03-04,2003,3,4
3,63,0.000000,2003-03-05,2003,3,5
4,64,0.000077,2003-03-06,2003,3,6


In [ ]:
precipitation_df.dtypes

DOY                       int64
precipitation           float64
Timestamp        datetime64[ns]
Year                      int64
Month                     int64
Day                       int64
dtype: object

In [ ]:
precipitation_df.shape

(7276, 6)

In [ ]:
average_precipitation_per8 = []
for i in range(len(precipitation_df)):
  if i < 8:
    x = precipitation_df.iloc[:i,:].precipitation.mean()
  else:
    x = precipitation_df.iloc[i-8:i,:].precipitation.mean()

  average_precipitation_per8.append(x)

In [ ]:
len(average_precipitation_per8)

7276

In [ ]:
precipitation_df['average_per8'] = average_precipitation_per8

In [ ]:
precipitation_df = precipitation_df[precipitation_df['DOY'].isin(list(np.arange(0, 368, 8).tolist()))].reset_index(drop=True)

In [ ]:
alt.Chart(precipitation_df).mark_bar(size=1).encode(
    x='Timestamp:T',
    y='average_per8:Q',
    color=alt.Color(
        'average_per8:Q', scale=alt.Scale(scheme='turbo', domain=(precipitation_df.average_per8.min(),precipitation_df.average_per8.max()))),
    tooltip=[
        alt.Tooltip('Timestamp:T', title='Date'),
        alt.Tooltip('average_per8:Q', title='average_per8')
    ]).properties(width=1200, height=600)

alt.Chart(...)

In [ ]:
highlight = alt.selection(
    type='single', on='mouseover', fields=['Year'], nearest=True, bind='legend')
color = alt.condition(highlight,
                      alt.Color('Year:O', title='Year'),
                      alt.value('lightgray'))

base = alt.Chart(precipitation_df).encode(
    x=alt.X('DOY:Q', scale=alt.Scale(domain=[0, 353], clamp=True)),
    y=alt.Y('average_per8:Q', scale=alt.Scale(domain=[precipitation_df.average_per8.min(),precipitation_df.average_per8.max()])),
    color=color)

points = base.mark_circle().encode(
    opacity=alt.value(0),
    tooltip=[
        alt.Tooltip('Year:O', title='Year'),
        alt.Tooltip('Timestamp:T', title='DOY'),
        alt.Tooltip('average_per8:Q', title='average_per8')
    ]).add_selection(highlight)

lines = base.mark_line().encode(
    size=alt.condition(~highlight, alt.value(1), alt.value(3)))

(points + lines).properties(width=1200, height=600)

alt.LayerChart(...)

In [ ]:
precipitation_df.head()

,DOY,precipitation,Timestamp,Year,Month,Day,average_per8
0,64,0.000077,2003-03-06,2003,3,6,0.000000
1,72,0.000000,2003-03-14,2003,3,14,0.001151
2,80,0.000000,2003-03-22,2003,3,22,0.000000
3,88,0.000000,2003-03-30,2003,3,30,0.000000
4,96,0.000000,2003-04-07,2003,4,7,0.000000


In [ ]:
mean_DOY = precipitation_df.groupby('DOY').mean()[['precipitation']]
mean_DOY.head()

,precipitation
DOY,
0,0.011701
8,0.007251
16,0.006839
24,0.002565
32,0.002113


In [ ]:
mean_DOY[mean_DOY.index == 0]['precipitation'][0]

0.011700690792565614

In [ ]:
difference_list = []
for i in range(len(precipitation_df)):
  DOY = precipitation_df['DOY'][i]
  precipitation_value = precipitation_df['precipitation'][i]
  difference = precipitation_value - mean_DOY[mean_DOY.index == DOY]['precipitation'][DOY]
  difference_list.append(difference)

In [ ]:
precipitation_df['difference'] = difference_list

In [ ]:
precipitation_df.head()

,DOY,precipitation,Timestamp,Year,Month,Day,difference
0,64,0.000000,2003-03-06,2003,3,6,-0.000464
1,72,0.001151,2003-03-14,2003,3,14,0.000536
2,80,0.000000,2003-03-22,2003,3,22,-0.000658
3,88,0.000000,2003-03-30,2003,3,30,-0.000735
4,96,0.000000,2003-04-07,2003,4,7,-0.000400


In [ ]:
alt.Chart(precipitation_df).mark_bar(size=2).encode(
    x='Timestamp:T',
    y='difference:Q',
    color=alt.condition(
        predicate=alt.datum.difference >= 0,
        if_true=alt.value("blue"),
        if_false=alt.value("red")
    ),
    tooltip=[
        alt.Tooltip('Timestamp:T', title='Date'),
        alt.Tooltip('difference:Q', title='Difference')
    ]).properties(width=1200, height=600)

alt.Chart(...)

In [ ]:
precipitation_df = precipitation_df.sort_values(by='Timestamp').reset_index(drop=True)

In [ ]:
precipitation_df.iloc[12:,:].shape

(904, 7)

In [ ]:
average_precipitation = []
average_difference = []

for i in range(12,len(precipitation_df)):
  df = precipitation_df.iloc[i-12:i,:]
  average_precipitation.append(df.precipitation.mean())
  average_difference.append(df.difference.mean())

In [ ]:
len(average_precipitation)

904

In [ ]:
precipitation_df2 = precipitation_df.iloc[12:,:].copy()
precipitation_df2['average_precipitation'] = average_precipitation
precipitation_df2['average_difference'] = average_difference

In [ ]:
alt.Chart(precipitation_df2).mark_bar(size=3).encode(
    x='Timestamp:T',
    y='average_precipitation:Q',
    color=alt.Color(
        'average_precipitation:Q', scale=alt.Scale(scheme='turbo', domain=(precipitation_df2.average_precipitation.min(),precipitation_df2.average_precipitation.max()))),
    tooltip=[
        alt.Tooltip('Timestamp:T', title='Date'),
        alt.Tooltip('average_precipitation:Q', title='average_precipitation')
    ]).properties(width=1200, height=600)

alt.Chart(...)

In [ ]:
alt.Chart(precipitation_df2).mark_bar(size=2).encode(
    x='Timestamp:T',
    y='average_difference:Q',
    color=alt.condition(
        predicate=alt.datum.average_difference >= 0,
        if_true=alt.value("blue"),
        if_false=alt.value("red")
    ),
    tooltip=[
        alt.Tooltip('Timestamp:T', title='Date'),
        alt.Tooltip('average_difference:Q', title='average_difference')
    ]).properties(width=1200, height=600)

alt.Chart(...)

In [ ]:
outputpath = '/content/drive/MyDrive/ND/' + country_name+ '.csv'
precipitation_df2.to_csv(outputpath, index=False)